Part 2.
Data Wrangling and Exploratory Data Analysis (EDA)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

df = pd.read_csv('/content/real_estate_apart_data.csv', sep = ',')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4056 entries, 0 to 4055
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Status        4056 non-null   object 
 1   Price         4056 non-null   object 
 2   Description   4056 non-null   object 
 3   Location      4056 non-null   object 
 4   Area          4048 non-null   object 
 5   No_rooms      4052 non-null   float64
 6   No_bathrooms  4019 non-null   float64
dtypes: float64(2), object(5)
memory usage: 221.9+ KB


In [ ]:
df.head(10)

,Status,Price,Description,Location,Area,No_rooms,No_bathrooms
0,Rent,300,2+1 APARTMENT FOR RENT IN ILIRIA BEACH!!,"Durrës, Albania",95,2.0,1.0
1,Rent,350,APARTMENT 1+1 FOR RENT,"Vlorë, Albania",90,1.0,1.0
2,Sale,"78,200",1+1 APARTMENT FOR SALE IN MAL TE ROBIT!,"Durrës, Albania",68,1.0,1.0
3,Sale,"78,200",1+1 APARTMENT FOR SALE IN MAL TE ROBIT!,"Durrës, Albania",68,1.0,1.0
4,Rent,600,1+1 APARTMENT FOR RENT MUNICIPALITY OF PARIS!,"Kompleksi Kika 1 Tirana, Albania",70,1.0,1.0
5,Rent,"1,200",SUPER APARTMENT FOR RENT 2+1+2 NEAR THE BOTANI...,"Tirana, Albania",125,2.0,2.0
6,Sale,Price upon reque,APARTMENT FOR SALE 3+1+2 TOILET IN NEW BUILDIN...,"RRUGA MOSKAT Tirana, Albania",220,3.0,2.0
7,Sale,"310,000","APARTMENT 3+1+2 FOR SALE IN ""DRY LAKE""!","Liqeni i Thate Tirana, Albania",180,3.0,2.0
8,Rent,700,APARTMENT 2 +1+2 FOR RENT NEAR THE MUNICIPALIT...,"Tirana, Albania",97,2.0,2.0
9,Rent,"1,250",2+1 apartment for rent at LICEU,"Rruga Themistokli Germenji Tirana, Albania",120,2.0,2.0


Here's a short info about the columns:

Status -> Status of the property, On sale, For rent, Reserved, etc.

Price -> Total price in Euro for the propery on sale/rent

Description -> Long description of the object being on sale/rent, sometimes including location.

Location -> City of the property, sometimes also information about the street or neighbourhood

Area -> Area in m2 (Some missing values)

No_rooms -> No. of rooms in the property (We have some missing values here, and probably some wrong ones from the scraping)

No_bathrooms -> No. of bathrooms in the property (We have some missing values here, and probably some wrong ones from the scraping)


As we can see from info and head method, we have 4056 entries with 7 features. Now, let's start the data cleaning process. First of all, I will discuss the Location column, as the famous real-estate quote goes: "Location, location, location." The Location column contains the city and country (ofc always Albania) of the property, but sometimes it also contains a very important information about the address of the property. This last bit of information is sometimes included in the Description column. The Location column will store only the address (if available) and another column will be added to the dataframe, storing the City.

In [ ]:
df['City'] = df['Location'].apply(lambda x: x.split()[-2])
df['City'] = df['City'].str.replace(',', '')
df['Location'] = df['Location'].apply(lambda x: ' '.join(x.split()[0:-2]))

df['City'].value_counts()


Tirana          2292
Durrës          1201
Vlorë            203
Sarandë          147
Shëngjin          75
Shkodër           46
Korçë             40
Kavajë            16
Kamëz             12
Pogradec           9
Lezhë              4
Lushnje            3
Fier               3
Elbasan            3
Krujë              1
Sarand&euml;       1
Name: City, dtype: int64

There are cities with less than 10 properties, which makes a small number considering there will also be a split in rented and on sale. So, it's better to remove these rows.

In [ ]:
cities_to_remove = ['Pogradec', 'Lezhë', 'Lushnje', 'Fier', 'Elbasan', 'Krujë', 'Sarand&euml;']
df = df[~df['City'].isin(cities_to_remove)]

In [ ]:
df['Price'].value_counts()

Price upon reque    359
500                  66
400                  66
600                  65
300                  59
                   ... 
128,900               1
131,220               1
104,700               1
62,100                1
26,500                1
Name: Price, Length: 968, dtype: int64

When checking the Price column with the value_counts method, we see a long string ('Price upon reque') meaning that the price for some properties is available only when contacting the respective agent. So, without the price, these rows become meaningless and removing them is the only option left.

In [ ]:
df = df[df['Price'] != 'Price upon reque']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3673 entries, 0 to 4054
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Status        3673 non-null   object 
 1   Price         3673 non-null   object 
 2   Description   3673 non-null   object 
 3   Location      3673 non-null   object 
 4   Area          3669 non-null   object 
 5   No_rooms      3671 non-null   float64
 6   No_bathrooms  3641 non-null   float64
 7   City          3673 non-null   object 
dtypes: float64(2), object(6)
memory usage: 258.3+ KB


We have only the priced apartaments in our dataframe, 3546 entries. Now,  let's select from the dataframe only the rows which have the no_bathrooms less than or equal to 3.  

In [ ]:
df['No_bathrooms'].value_counts()

1.0       2183
2.0       1347
3.0         73
4.0          8
5.0          5
6.0          3
12.0         2
14.0         1
332.0        1
7.0          1
173.0        1
192.0        1
1096.0       1
216.0        1
260.0        1
604.0        1
9.0          1
139.0        1
114.0        1
121.0        1
55.0         1
211.0        1
46.0         1
10.0         1
50.0         1
82.0         1
3410.0       1
Name: No_bathrooms, dtype: int64

In [ ]:
df['No_rooms'].value_counts()

2.0      1756
1.0      1322
3.0       491
4.0        46
5.0        13
6.0         4
12.0        2
7.0         2
318.0       1
302.0       1
169.0       1
191.0       1
155.0       1
149.0       1
319.0       1
410.0       1
221.0       1
344.0       1
707.0       1
340.0       1
197.0       1
227.0       1
569.0       1
398.0       1
299.0       1
806.0       1
463.0       1
67.0        1
100.0       1
43.0        1
87.0        1
55.0        1
14.0        1
8.0         1
86.0        1
239.0       1
138.0       1
18.0        1
688.0       1
441.0       1
10.0        1
203.0       1
845.0       1
Name: No_rooms, dtype: int64

Now, let's also remove the properties which have more than 4 rooms. This could potentially remove any luxury apartament with more rooms.

In [ ]:
df = df[df['No_rooms'] <= 4]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3615 entries, 0 to 4054
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Status        3615 non-null   object 
 1   Price         3615 non-null   object 
 2   Description   3615 non-null   object 
 3   Location      3615 non-null   object 
 4   Area          3613 non-null   object 
 5   No_rooms      3615 non-null   float64
 6   No_bathrooms  3615 non-null   float64
 7   City          3615 non-null   object 
dtypes: float64(2), object(6)
memory usage: 254.2+ KB


In [ ]:
df= df[df['No_bathrooms'] <= 3]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3592 entries, 0 to 4053
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Status        3592 non-null   object 
 1   Price         3592 non-null   object 
 2   Description   3592 non-null   object 
 3   Location      3592 non-null   object 
 4   Area          3592 non-null   object 
 5   No_rooms      3592 non-null   float64
 6   No_bathrooms  3592 non-null   float64
 7   City          3592 non-null   object 
dtypes: float64(2), object(6)
memory usage: 252.6+ KB


The above 'tricks' looks that achieved our objective. Now we have a dataframe with 3467 non-null entries which includes only apartaments (for rent or on sale) in Albania.

In [ ]:
df['Status'].value_counts()

 Sale                2628
Rent                  766
erved                 184
al.in_negotiation      14
Name: Status, dtype: int64

The Status column does not have null values, but instead it has a small problem. The third (Reserved property) and fourth (in_negotiation) category have to be converted to On Sale and For Rent, because we're not interested if the property is reserved or already sold.
Here, the workaround could be to check the price column and identify what could be a threshold price that could separate sale prices from rent prices. I think that by choosing a threshhold of 6000 euros we are rather safe. But first, we have to remove the comma character in the price column and then convert the column to integer.



In [ ]:
df['Price'] = df['Price'].str.replace(',', '')
df['Price'] = df['Price'].astype(int)

In [ ]:
threshold = 6000  # Example threshold value 6000 euros.

# Create a new 'Category' column based on the threshold
df['Category'] = df['Price'].apply(lambda price: 'Sale' if price > threshold else 'Rent')


In [ ]:
df['Category'].value_counts()

Sale    2781
Rent     811
Name: Category, dtype: int64

Let's have a sanity check and look at the maximum price for the Rent Status and the minimum price for the Sale category.

In [ ]:
max_price_for_rent = df[df['Category'] == 'Rent']['Price'].max()
min_price_for_sale = df[df['Category'] == 'Sale']['Price'].min()

print(f"Maximum rent price is {max_price_for_rent} Euros")
print(f"Minimum sale price is {min_price_for_sale} Euros")

Maximum rent price is 5000 Euros
Minimum sale price is 14000 Euros


As we can see from the output, maybe there's something wrong with the maximum rent price.

In [ ]:
max_price_index_for_rent = df[df['Category'] == 'Rent']['Price'].idxmax()
max_price_row_for_rent = df.loc[max_price_index_for_rent]

print(max_price_row_for_rent)

Status                             Rent
Price                              5000
Description     3+1 apartment for rent!
Location            Kompleksi "Halisof"
Area                                130
No_rooms                            3.0
No_bathrooms                        2.0
City                             Tirana
Category                           Rent
Name: 493, dtype: object


By checking the Description of the property, it is okay as it looks like a luxury apartment in a newly build residence.

In [ ]:
min_price_index_for_rent = df[df['Category'] == 'Rent']['Price'].idxmin()
min_price_row_for_rent = df.loc[min_price_index_for_rent]

print(min_price_row_for_rent)

Status                                                       Rent
Price                                                          50
Description     APARTMENTS FOR DAILY RENT IN THE CENTER OF TIR...
Location                                      Rruga Kajo Karafili
Area                                                           39
No_rooms                                                      1.0
No_bathrooms                                                  1.0
City                                                       Tirana
Category                                                     Rent
Name: 1242, dtype: object


AS suspected, this is the price for daily rent, so let's remove it, and rerun the previous code. Now, we have to remove the entries that are daily rents, by removing rows that have a substring 'DAILY' in the Description column. Now, I suspect there are other substring to add, such as 'AIR', 'BnB', and I will also add 'VILLA' in case there might b

In [ ]:
import regex as re

substrings_to_remove = ['DAILY', 'AIR', 'BnB']

# Create a boolean mask that checks if any of the substrings are present in the 'Description' column
mask = df['Description'].str.contains('|'.join(map(re.escape, substrings_to_remove)), case=False, regex=True)

# Use the mask to filter the DataFrame and keep only rows that don't contain any of the substrings
df = df[~mask]

Let's drop the 'Status' column. Convert the Area column to integer and
then create a new column, Price per sq meter, which will be valuable for further analysis.

In [ ]:
df = df.drop('Status', axis=1)

In [ ]:
df['No_rooms'] = df['No_rooms'].astype(int)
df['No_bathrooms'] = df['No_bathrooms'].astype(int)

In [ ]:
df['Area'] = df['Area'].str.replace(',', '.').astype(float)


In [ ]:
df['Price/m2'] = df['Price'] / df['Area']

In [ ]:
df.head(8)

,Price,Description,Location,Area,No_rooms,No_bathrooms,City,Category,Price/m2
0,300,2+1 APARTMENT FOR RENT IN ILIRIA BEACH!!,,95.0,2,1,Durrës,Rent,3.157895
1,350,APARTMENT 1+1 FOR RENT,,90.0,1,1,Vlorë,Rent,3.888889
2,78200,1+1 APARTMENT FOR SALE IN MAL TE ROBIT!,,68.0,1,1,Durrës,Sale,1150.000000
3,78200,1+1 APARTMENT FOR SALE IN MAL TE ROBIT!,,68.0,1,1,Durrës,Sale,1150.000000
4,600,1+1 APARTMENT FOR RENT MUNICIPALITY OF PARIS!,Kompleksi Kika 1,70.0,1,1,Tirana,Rent,8.571429
5,1200,SUPER APARTMENT FOR RENT 2+1+2 NEAR THE BOTANI...,,125.0,2,2,Tirana,Rent,9.600000
7,310000,"APARTMENT 3+1+2 FOR SALE IN ""DRY LAKE""!",Liqeni i Thate,180.0,3,2,Tirana,Sale,1722.222222
8,700,APARTMENT 2 +1+2 FOR RENT NEAR THE MUNICIPALIT...,,97.0,2,2,Tirana,Rent,7.216495
